In [12]:
#from __future__ import division
import pandas as pd
import re
import numpy as np
import datetime

In [13]:
import urllib
import urllib.request

In [14]:
from bs4 import BeautifulSoup 

In [15]:
def download_csv(url, file):
    urllib.request.urlretrieve(url, file)

In [16]:
def read_stock_csv(surl):
    df = pd.read_csv(surl, parse_dates=1)
    df['Avg Price'] = ((df['High'] + df['Low']) / 2).round(4)
    return df

In [17]:
def read_div_csv(durl):
    #durl = 'http://www.aastocks.com/en/stocks/analysis/dividend.aspx?symbol=00388'
    page = urllib.request.urlopen(durl)
    soup = BeautifulSoup(page, 'lxml')

    # web scraping
    data = []
    table = soup.find('table', attrs={'class':'cnhk-cf tblM s4 s5'})
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])

    # convert to dataframe
    df = pd.DataFrame(data[1:], columns=data[0])
    
    return df

In [18]:
# extract the dividend amount from text field
def ext_div(text):
    loc = str(text).find('HKD')
    if loc != -1:
        return float(text[loc+4:loc+4+6])
    else:
        loc = str(text).find('CAD')
        if loc != -1:
            return float(text[loc+4:loc+4+6])*6.18
        else:
            loc = str(text).find('RMB')
            if loc != -1:
                return float(text[loc+4:loc+4+6])*1.19
            else:
                loc = str(text).find('USD')
                if loc != -1:
                    return float(text[loc+4:loc+4+6])*7.8
                else:
                    return text

In [19]:
def months_between(start, end):
    start_year = int(start[:start.find('-')])
    start_month = int(start[start.find('-')+1:])
    end_year = int(end[:end.find('-')])
    end_month = int(end[end.find('-')+1:])
    
    year_diff = end_year - start_year
    month_diff = end_month - start_month
    
    diff = []
    
    if year_diff <= 0:
        for j in range(start_month, end_month + 1):
            diff.append(str(start_year).rjust(2, '0') + '-' + str(j).rjust(2, '0'))
    else:
        for i in range(start_year, end_year + 1):
            if (i == start_year ):
                for j in range(start_month, 12 + 1):
                    diff.append(str(i).rjust(2, '0') + '-' + str(j).rjust(2, '0'))
            elif (i == end_year):
                for j in range(1, end_month + 1):
                    diff.append(str(i).rjust(2, '0') + '-' + str(j).rjust(2, '0'))
            else:
                for j in range(1, 12 + 1):
                    diff.append(str(i).rjust(2, '0') + '-' + str(j).rjust(2, '0'))                    
      
    return diff

In [20]:
def buy_shares(results, contrib, fee):
    shares = []
    total_shares = 0
    total_contrib = 0
    f = 0
    s = 0
    
    for r in results:
        s = int((contrib - fee)/r[1])
        shares.append([r[0], s])
        total_shares += s
        total_contrib += round(s * r[1], 4)
        f += fee
        
    shares = pd.DataFrame(shares, columns=['Date', 'No of shares'])
    
    return shares, total_shares, total_contrib, f

In [21]:
def cal_dividend(shares, ddf):
    ddf['Particular'] = ddf['Particular'].apply(ext_div)
    ddf['Ex-Date'] = ddf['Ex-Date'].str.replace('/', '-')
    temp = ddf[ddf['Type'].str.contains('Cash')]
        
    earliest = shares['Date'].min()
    latest = shares['Date'].max()
    temp = temp[(temp['Ex-Date'] <= latest) & (temp['Ex-Date'] >= earliest)]

    dividend = 0
    accumulated_shares = 0
    
    for row in temp.itertuples():   
        s = shares[(row[6] >= shares['Date']) & (row[6] >= earliest)].sum().iloc[1]
        dividend += row[4] * s
        
    return dividend

In [22]:
def cal_profit(total_shares, total_contrib, current_price, dividend=0):
    profit = (current_price * total_shares + dividend - total_contrib) / total_contrib
    
    return profit    

In [23]:
def eval_stock(surl, durl, contrib, fee, start, end, day=10):
    # contrib: monthly contribution amount
    # fee
    # current_price
    # start: a string indicate the start year and month
    # end: a string indicate the end year and month
    # day: day to execution    
    
    results = []
    today = datetime.datetime.now().timetuple()
    
    #durl = 'http://www.aastocks.com/en/stocks/analysis/dividend.aspx?symbol='+str(stock_id).rjust(5, '0')
    #surl = 'http://real-chart.finance.yahoo.com/table.csv?s='+stock_id+'.HK'+'&a=05&b=16&c=2000&d=03&e='+str(today[2])+'&f=2016&g=d&ignore=.csv
    
    months = months_between(start, end)
    
    df = read_stock_csv(surl)
    ddf = read_div_csv(durl)
    
    # current price is the latest average price
    current_price = df[df['Date'] ==np.max(df['Date'])].iloc[0]['Avg Price']
    
    for m in months:
        for d in range(day, day + 5):
            temp = df[df['Date'].str.contains(m + '-' + str(d).rjust(2, '0'))]
            if len(temp) > 0:
                results.append([m + '-' + str(d).rjust(2, '0'), temp.iloc[0]['Avg Price']])
                break
                
    shares, total_shares, total_contrib, total_fee = buy_shares(results, contrib, fee)
    dividend = cal_dividend(shares, ddf)
    
    profit = cal_profit(total_shares, total_contrib, current_price, dividend)
                    
    return profit

In [24]:
stocks = ['0001', '0002', '0003', '0004', '0005', '0006', 
          '0011', '0012', '0016', '0017', '0019', 
          '0023', '0027', '0066', '0083', '0097',
          '0101', '0119', '0123', '0135', '0144', '0151',
          '0165', '0168', '0175', '0267', '0291', '0293', 
          '0322', '0323', '0330', '0358', '0388', '0390',
          '0392', '0489', '0494', '0506', '0669', '0688',
          '0700', '0728', '0753', '0762', '0763', '0813',
          '0823', '0836', '0857', '0883', '0902', '0914',
          '0916', '0939', '0941', '0945', '0991', '0992',
          '0998', '1038', '1044', '1066', '1088', '1099',
          '1109', '1113', '1114', '1157', '1171', '1193',
          '1211', '1288', '1299', '1313', '1336', '1339',
          '1398', '1800', '1880', '1898', '1919', '1928',
          '1929', '1988', '2018', '2038', '2238', '2318',
          '2319', '2328', '2333', '2338', '2388', '2600',
          '2601', '2628', '2800', '2822', '2823', '2827',
          '2828', '2833', '2866', '2883', '2888', '2899',
          '3049', '3323', '3328', '3333', '3968', '3988', 
          '6030', '6837'        
          ]

In [25]:
#today = datetime.datetime.now().timetuple()

# download csv files
#for s in stocks:
#    surl = 'http://real-chart.finance.yahoo.com/table.csv?s='+s+'.HK'+'&a=05&b=16&c=2000&d=03&e='+str(today[2])+'&f=2016&g=d&ignore=.csv'
#    download_csv(surl, s+'.csv')
#http://real-chart.finance.yahoo.com/table.csv?s=2800.HK&a=11&b=31&c=2000&d=03&e=26&f=2016&g=v&ignore=.csv    

In [33]:
results = []
for s in stocks:
    surl = s+'.csv'
    durl = 'http://www.aastocks.com/en/stocks/analysis/dividend.aspx?symbol='+str(s).rjust(5, '0')
    results.append([s, eval_stock(surl, durl, 500, 50, '2014-01', '2016-04')])
    #results = eval_stock(surl, durl, 2000, 50, '2015-01', '2016-04')

array([['0.0489852092086', '0001'],
       ['0.124817686552', '0002'],
       ['-0.181414855643', '0003'],
       ['-0.181335015097', '0004'],
       ['-0.248002917822', '0005'],
       ['0.0217347597842', '0006'],
       ['0.0842948916372', '0011'],
       ['-0.0882978302789', '0012'],
       ['-0.0755364431911', '0016'],
       ['-0.10782887339', '0017'],
       ['-0.0782892247577', '0019'],
       ['-0.00868792998299', '0023'],
       ['-0.239333434394', '0027'],
       ['0.196132224164', '0066'],
       ['0.0720330130418', '0083'],
       ['0.0452938395106', '0097'],
       ['-0.262457422267', '0101'],
       ['-0.293712410928', '0119'],
       ['-0.188102911514', '0123'],
       ['-0.18875938134', '0135'],
       ['-0.093280516307', '0144'],
       ['-0.278125686828', '0151'],
       ['0.07932283377', '0165'],
       ['-0.366267685465', '0168'],
       ['0.144460908543', '0175'],
       ['-0.0937530145412', '0267'],
       ['0.277086104066', '0291'],
       ['-0.109803088813', '02

In [38]:
results = pd.DataFrame(results, columns=['stock', 'profit'])

In [40]:
results.sort(columns='profit', ascending=False)

C:\Users\home\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,stock,profit
109,3333,0.652841
84,2018,0.315050
59,1038,0.307969
63,1099,0.291794
77,1800,0.277914
26,0291,0.277086
38,0669,0.210300
105,2899,0.201761
13,0066,0.196132
40,0700,0.179294
